In [ ]:
import subprocess
import os
import argparse
from pydub import AudioSegment
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import csv
import sys
import re
from scipy.stats import truncnorm, zscore
import numpy as np
import pandas as pd

# Faster inference with TF32
torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cudnn.allow_tf32 = False

# Sentence level transcription
import whisperx
import nltk
from nltk.tokenize import sent_tokenize

# Define upper & lower bounds for audio in seconds
MIN_SEC, MAX_SEC = 3, 11.6

# Define WhisperX parameters
BATCH_SIZE = 16
COMPUTE_TYPE = "float16"
WHISPER_MODEL = "small" # Options: tiny, base, small, medium, large-v2, large-v3

def yt_download(url):
    """
    Downloads a wav file from given YouTube url.
    """
    # Create audio folder
    audio_dir = f"./audio/"
    if not os.path.exists(audio_dir):
        os.makedirs(audio_dir)

    # Get video title (used later to create dir)
    result = subprocess.run(
        ["yt-dlp", "--get-title", url],
        capture_output=True, text=True
    )
    title = result.stdout.strip()

    # Clean title by removing special characters
    title = re.sub(r'[^a-zA-Z0-9_\- ]', '', title)

    yt_audio_path = os.path.join(audio_dir, f"{title}.wav")

    subprocess.run([
        "yt-dlp",
        "-f", "bestaudio",
        "--extract-audio",
        "--audio-format", "wav", # convert to wav as this is XTTS prefered format
        "-o", yt_audio_path,
        url
    ])

    return yt_audio_path, title

def load_and_concat(file_paths, title, normalize_dbfs=-20.0):
    # Create audio folder
    audio_dir = f"./audio/"
    if not os.path.exists(audio_dir):
        os.makedirs(audio_dir)
    
    output_path = os.path.join("audio", f"{title}.wav")

    combined = AudioSegment.from_file(file_paths)

    # Normalize loudness
    print(f"Combined loudness before normalization: {combined.dBFS:.2f} dBFS")
    change_dBFS = normalize_dbfs - combined.dBFS
    combined = combined.apply_gain(change_dBFS)
    print(f"Normalized loudness to {normalize_dbfs} dBFS")

    combined.export(output_path, format="wav")

    return output_path

def resample_audio(audio_path):
    audio = AudioSegment.from_file(audio_path)

    frame_rate = audio.frame_rate
    channels = audio.channels
    sample_width_bits = audio.sample_width * 8

    print(f"Original audio: {frame_rate} Hz, {channels} channel(s), {sample_width_bits}-bit")

    if frame_rate != 22050 or channels != 1 or sample_width_bits != 16:
        print("Resampling to 22050 Hz, mono, 16-bit PCM.")
        resampled = audio.set_frame_rate(22050).set_channels(1).set_sample_width(2)
        resampled.export(audio_path, format="wav")
        print("Resampling complete and file overwritten.")
    else:
        print("Audio already meets the required format.")

    return audio_path

def chunk_sentences(
    audio_path,
    title,
    min_sec: float = MIN_SEC,
    max_sec: float = MAX_SEC,
    char_limit: int = 250,
    compute_type: str = COMPUTE_TYPE,
    batch_size: int = BATCH_SIZE,
    whisper_model: str = WHISPER_MODEL,
):
    dataset_dir = os.path.join("./datasets", title)
    wavs_dir     = os.path.join(dataset_dir, "wavs")
    os.makedirs(wavs_dir, exist_ok=True)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    # WhisperX transcription + alignment
    print("Loading model and params...")
    model      = whisperx.load_model(whisper_model, device=device, compute_type=compute_type)
    audio_arr  = whisperx.load_audio(audio_path)
    base       = model.transcribe(audio_arr, batch_size=batch_size, language="en")
    align_mdl, meta = whisperx.load_align_model("en", device)
    aligned    = whisperx.align(base["segments"], align_mdl, meta, audio_arr,
                                device, return_char_alignments=False)

    word_segs  = aligned["word_segments"]
    full_text  = " ".join(w["word"] for w in word_segs)
    sentences  = sent_tokenize(full_text)

    # map sentences -> (start, end)
    print("Mapping chunks")
    sent_segs, idx = [], 0
    for sent in sentences:
        n = len(sent.split())
        if idx + n > len(word_segs):
            break
        sent_segs.append({
            "text":  sent,
            "start": word_segs[idx]["start"],
            "end":   word_segs[idx + n - 1]["end"],
        })
        idx += n

    # Build chunks respecting all limits
    print("Building chunks")
    chunks, buf, dur, txt = [], [], 0.0, ""
    for seg in sent_segs:
        seg_dur  = seg["end"] - seg["start"]
        seg_text = seg["text"]
        # Case 1: sentence is valid and fits limits
        if min_sec <= seg_dur <= max_sec and len(seg_text) <= char_limit:
            if buf:
                chunks.append(buf)
                buf, dur, txt = [], 0.0, ""
            chunks.append([seg])
            continue

        # Case 2: add sentence to buffer
        if dur + seg_dur > max_sec or len(txt) + 1 + len(seg_text) > char_limit:
            if dur >= min_sec:
                chunks.append(buf)
                buf, dur, txt = [], 0.0, ""
            else:
                # buffer too short
                pass

        buf.append(seg)
        dur += seg_dur
        txt += (" " if txt else "") + seg_text

    if buf:
        chunks.append(buf)

    # Write wavs + metadata
    audio               = AudioSegment.from_file(audio_path)
    metadata_path       = os.path.join(dataset_dir, "metadata.csv")
    with open(metadata_path, "w", encoding="utf‑8‑sig") as meta_f:
        for i, chunk in enumerate(chunks):
            start  = int(chunk[0]["start"] * 1000)
            end    = int(chunk[-1]["end"] * 1000)
            if end <= start:
                continue
            piece  = audio[start:end]
            if len(piece) < min_sec*1000 or len(piece) > max_sec*1000:
                continue # double‑check duration
            fname  = f"chunk_{i:04}.wav"
            piece.export(os.path.join(wavs_dir, fname), format="wav")

            text   = " ".join(c["text"] for c in chunk)
            if len(text) > char_limit: # final text safety check
                # Truncate at nearest word before limit
                text = text[:char_limit].rsplit(" ", 1)[0] + " …"

            # Write LJSpeech‑style row: <id>|<text>|<text>
            meta_f.write(f"{os.path.splitext(fname)[0]}|{text}|{text}\n")

    return dataset_dir, wavs_dir

def main():
    title = "noramlized_personal_voice"
    audio_input = "./audio/set_1_123_min.wav"
    
    url = None
    
    if url:
        print('Downloading YouTube video...')
        audio_path, title = yt_download(url)
        print(f'YouTube video: {title} downloaded.')
    else:
        print('Loading and concatenating input files.')
        audio_path = load_and_concat(audio_input, title)
        print(f"Audio written to: {audio_path}")

        print('Resampling audio to 24kHz.')
        audio_path = resample_audio(audio_path)
        print('Audio resampling complete.')

        print("Chunking by sentence using WhisperX.")
        dataset_dir, wavs_dir = chunk_sentences(audio_path, title)
        print(f"Chunks and metadata saved in {dataset_dir}")

if __name__ == "__main__":
    main()

C:\Users\12017\Desktop\NJIT\DS677_852_Project\src\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading and concatenating input files.
Combined loudness before normalization: -30.58 dBFS
Normalized loudness to -20.0 dBFS
Audio written to: audio\noramlized_personal_voice.wav
Resampling audio to 24kHz.
Original audio: 22050 Hz, 2 channel(s), 16-bit
Resampling to 22050 Hz, mono, 16-bit PCM.
Resampling complete and file overwritten.
Audio resampling complete.
Chunking by sentence using WhisperX.
Using device: cuda
Loading model and params...


C:\Users\12017\Desktop\NJIT\DS677_852_Project\src\venv\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\12017\.cache\huggingface\hub\models--Systran--faster-whisper-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qb

No language specified, language will be first be detected for each audio file (increases inference time).
>>Performing voice activity detection using Pyannote...
Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.6.0+cu124. Bad things might happen unless you revert torch to 1.x.


C:\Users\12017\Desktop\NJIT\DS677_852_Project\src\venv\lib\site-packages\pyannote\audio\utils\reproducibility.py:74: ReproducibilityWarning: TensorFloat-32 (TF32) has been disabled as it might lead to reproducibility issues and lower accuracy.
It can be re-enabled by calling
   >>> import torch
   >>> torch.backends.cuda.matmul.allow_tf32 = True
   >>> torch.backends.cudnn.allow_tf32 = True
See https://github.com/pyannote/pyannote-audio/issues/1370 for more details.

  warnings.warn(
